# Import packages


In [1]:
import argparse
import numpy as np
import os
import sys
import glog
import time

# If CPU is used, cpu extension might be needed

In [ ]:
#--cpu_extension /home/allen/R5/intel/computer_vision_sdk_2018.5.445/deployment_tools/inference_engine/lib/ubuntu_16.04/intel64/libcpu_extension_sse4.so 

# Add library path

In [2]:
sys.path.insert(0, "/home/acer/intel/computer_vision_sdk_2018.5.455/python/python3.6/ubuntu16")
from openvino.inference_engine import IENetwork, IEPlugin

# Loading model
set the path to model xml file

In [10]:
model_xml = "/home/allen/Documents/FACEID_TEST/FP32/pyramidnet_20190227.xml"
model_bin = os.path.splitext(model_xml)[0] + ".bin"
plugin = IEPlugin(device="CPU", plugin_dirs=None)

# Read IR
glog.info("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork(model=model_xml, weights=model_bin)

I0729 13:08:47.773116 10498 <ipython-input-10-fd935bd8ca8b>:6] Loading network files:
	/home/allen/Documents/FACEID_TEST/FP32/pyramidnet_20190227.xml
	/home/allen/Documents/FACEID_TEST/FP32/pyramidnet_20190227.bin


# Get model output name
Modification is needed if necessary

In [11]:
out_blob = next(iter(net.outputs))
glog.info(out_blob)

I0729 13:08:49.168881 10498 <ipython-input-11-3a1b9b625dfc>:2] fc1


# Get input shape by input name
Modification is needed if necessary

In [15]:
n, c, h, w = net.inputs['data1'].shape
glog.info("{} {} {} {}".format(n,c,h,w))

I0729 13:09:26.431843 10498 <ipython-input-15-c5ae25610b0d>:2] 1 3 112 112


# Set dummy input for inference

In [7]:
data = np.ones([1, 3, 112, 112])
data2 = np.zeros([1, 16, 56, 56])
data3 = np.zeros([1, 16, 28, 28])
data4 = np.zeros([1, 16, 14, 14])

exec_net = plugin.load(network=net)
del net

# Predict 
giving the dictionary of input to get the output by output blob name

In [16]:
output = exec_net.infer(inputs={'data1': data, 'data2': data2, 'data3': data3, 'data4': data4})[out_blob]
glog.info(output.shape)

I0729 13:09:32.353111 10498 <ipython-input-16-ac6122dd4cb8>:2] (1, 512)


# Command line version

In [3]:
import argparse
import numpy as np
import os
import sys
import glog
import time

sys.path.insert(0, "/home/acer/intel/computer_vision_sdk_2018.5.455/python/python3.6/ubuntu16")
from openvino.inference_engine import IENetwork, IEPlugin
def parse_arguments(argv):
    parser = argparse.ArgumentParser()
    parser.add_argument("-m", "--model", help="Path to an .xml file with a trained model.", required=True, type=str)
    parser.add_argument("-b", "--batch_size", help="Path to a folder with images or path to an image files", 
                        type=int)
    parser.add_argument("-l", "--cpu_extension",
                        help="MKLDNN (CPU)-targeted custom layers.Absolute path to a shared library with the kernels "
                             "impl.", type=str, default=None)
    parser.add_argument("-pp", "--plugin_dir", help="Path to a plugin folder", type=str, default=None)
    parser.add_argument("-d", "--device",
                        help="Specify the target device to infer on; CPU, GPU, FPGA or MYRIAD is acceptable. Sample "
                             "will look for a suitable plugin for device specified (CPU by default)", default="CPU",
                        type=str)
    parser.add_argument("--labels", help="Labels mapping file", default=None, type=str)
    parser.add_argument("-nt", "--number_top", help="Number of top results", default=10, type=int)
    parser.add_argument("-ni", "--number_iter", help="Number of inference iterations", default=1, type=int)
    parser.add_argument("-pc", "--perf_counts", help="Report performance counters", default=False, action="store_true")

    return parser.parse_args(argv)

args = parse_arguments(sys.argv[1:])
model_xml = args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)

# Read IR
glog.info("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork(model=model_xml, weights=model_bin)

out_blob = next(iter(net.outputs))
net.batch_size = args.batch_size

n, c, h, w = net.inputs['data'].shape

data = np.ones([1, 3, 112, 112])
data2 = np.zeros([1, 16, 56, 56])
data3 = np.zeros([1, 16, 28, 28])
data4 = np.zeros([1, 16, 14, 14])

exec_net = plugin.load(network=net)

output = exec_net.infer(inputs={'data': data, 'data2': data2, 'data3': data3, 'data4': data4})[out_blob]

glog.info(output.shape)

del net

usage: ipykernel_launcher.py [-h] -m MODEL [-b BATCH_SIZE] [-l CPU_EXTENSION]
                             [-pp PLUGIN_DIR] [-d DEVICE] [--labels LABELS]
                             [-nt NUMBER_TOP] [-ni NUMBER_ITER] [-pc]
ipykernel_launcher.py: error: the following arguments are required: -m/--model


SystemExit: 2

W0724 15:20:45.290234 14504 warnings.py:99] /home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)

